Installing Dash

In [1]:
#install if required-remove the hashtag below to perform installation
#%pip install dash

Importing the libraries

In [2]:
#Execute this only if installing other libraries are required-remove the hashtag below to perform installation
#%pip install plotly pandas


# Dashboard Code 

In [3]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

df_hist = pd.read_csv("BEV_ICE_BEVVans.csv")
df_hist['Year'] = df_hist['Quarter'].str[:4].astype(int)
df_hist['Qtr'] = df_hist['Quarter'].str[-1].astype(int)
df_hist = df_hist.sort_values(['Year', 'Qtr'])

df_bev_cars = pd.read_csv("Forecast_BEV_Cars_2025_2040.csv")
df_ICE = pd.read_csv("Forecast_ICE_Cars_2025_2040.csv")
df_vans = pd.read_csv("Forecast_BEVVans_2025_2040.csv")
df_forecast = pd.concat([df_bev_cars, df_ICE, df_vans], ignore_index=True)

def load_scenario(file, metric_name):
    df = pd.read_csv(file)
    if "Quarter" not in df.columns and "ds" in df.columns:
        df["Quarter"] = pd.to_datetime(df["ds"]).dt.to_period("Q").astype(str)
    df["Metric"] = metric_name
    return df[["Quarter", "Value", "Metric"]]

df_pol2021 = load_scenario("Forecast_BEVCars_Policy2021.csv", "BEV Sales - Policy 2021 Driven")
df_pol2425 = load_scenario("Forecast_BEVCars_Policy2024_2025.csv", "BEV Sales - Policy 2024 + 2025 Driven")
df_chargers = load_scenario("Forecast_BEVCars_ChargerDriven.csv", "BEV Sales - Charger Driven")
df_battery = load_scenario("Forecast_BEVCars_BatteryDriven.csv", "BEV Sales - Battery Price Driven")
df_scenarios = pd.concat([df_pol2021, df_pol2425, df_chargers, df_battery], ignore_index=True)

df_chargers_hist = pd.read_csv("Quarterly_Charging_Devices.csv")
df_chargers_hist = df_chargers_hist.rename(columns={"Total Charging Devices": "Value"})
df_chargers_hist["Metric"] = "Historical Chargers"
df_chargers_hist = df_chargers_hist[["Quarter", "Value", "Metric"]]

df_chargers_baseline = pd.read_csv("Forecast_Chargers_Baseline.csv")
df_chargers_baseline = df_chargers_baseline.rename(columns={"chargers_forecasted": "Value"})
df_chargers_baseline["Metric"] = "Baseline Forecast"
df_chargers_baseline = df_chargers_baseline[["Quarter", "Value", "Metric"]]

df_chargers_policy = pd.read_csv("Forecast_Chargers_Policy.csv")
df_chargers_policy = df_chargers_policy.rename(columns={"chargers_forecasted": "Value"})
df_chargers_policy["Metric"] = "2030 Target + BEV Sales Regressor Forecast"
df_chargers_policy = df_chargers_policy[["Quarter", "Value", "Metric"]]


# KPI
milestones = ["2030Q4", "2035Q4", "2040Q4"]

def kpi_block(df, metric, color):
    """Return KPI cards for milestone years for a single metric"""
    blocks = []
    for ms in milestones:
        match = df[(df["Metric"] == metric) & (df["Quarter"] == ms)]
        if not match.empty:
            value = int(match["Value"].values[0])
            blocks.append(html.Div([
                html.H4(ms, style={"textAlign": "center", "color": "yellow"}),
                html.P(f"{metric}: {value:,}",
                       style={"color": color, "textAlign": "center", "fontWeight": "bold"})
            ], style={
                "flex": "1", "margin": "10px", "padding": "15px",
                "backgroundColor": "#1b263b", "borderRadius": "12px",
                "boxShadow": "2px 2px 10px rgba(0,0,0,0.4)"
            }))
    return html.Div(blocks, style={"display": "flex", "justifyContent": "center"})


color_map_forecast = {
    "BEV Sales": "cyan",
    "ICE Sales - Fuel Price + 2030 Ban": "red",
    "BEV Vans Sales": "orange"
}

color_map_scenarios = {
    "BEV Sales - Policy 2021 Driven": "#32CD32",
    "BEV Sales - Policy 2024 + 2025 Driven": "orange",
    "BEV Sales - Charger Driven": "red",
    "BEV Sales - Battery Price Driven": "yellow"
}

color_map_chargers = {
    "Historical Chargers": "#FFFFFF",
    "Baseline Forecast": "#00BFFF",
    "2030 Target + BEV Sales Regressor Forecast": "#32CD32"
}


# dashboard layout
app = dash.Dash(__name__)
app.layout = html.Div([

    html.H2("UK EV Adoption Forecasting Dashboard",
            style={"color": "white", "textAlign": "center",
                   "fontSize": "30px", "fontFamily": "Trebuchet MS"}),

    # Section 1
    html.H4("Historical Vehicle Sales (2015–2024)", style={"color": "#FFD700", "textAlign": "center"}),
    dcc.Dropdown(id="metric-dropdown-hist",
                 options=[{"label": m, "value": m} for m in df_hist["Metric"].unique()],
                 value="BEV Sales", clearable=False,
                 style={"width": "40%", "margin": "auto"}),
    dcc.Graph(id="line-chart-hist"),

    # Section 2
    html.H4("Basic Forecasts (2025–2040): BEV Cars, Vans, and ICE Cars", style={"color": "#FFD700", "textAlign": "center"}),
    dcc.Dropdown(id="metric-dropdown-forecast",
                 options=[{"label": m, "value": m} for m in df_forecast["Metric"].unique()],
                 value="BEV Sales", clearable=False,
                 style={"width": "40%", "margin": "auto"}),
    dcc.Graph(id="line-chart-forecast"),
    html.Div(id="kpi-forecast"),

    # Section 3
    html.H4("Scenario Forecasts (2025–2040)", style={"color": "#FFD700", "textAlign": "center"}),
    dcc.Dropdown(id="metric-dropdown-scenarios",
                 options=[{"label": m, "value": m} for m in df_scenarios["Metric"].unique()],
                 value="BEV Sales - Policy 2021 Driven", clearable=False,
                 style={"width": "40%", "margin": "auto"}),
    dcc.Graph(id="line-chart-scenarios"),
    html.Div(id="kpi-scenarios"),

    # Section 4
    html.H4("Charger Forecasts (2022–2040)", style={"color": "#FFD700", "textAlign": "center"}),
    dcc.Dropdown(id="metric-dropdown-chargers",
                 options=[{"label": k, "value": k} for k in color_map_chargers.keys()],
                 value="Historical Chargers", clearable=False,
                 style={"width": "40%", "margin": "auto"}),
    dcc.Graph(id="line-chart-chargers"),
    html.Div(id="kpi-chargers")

], style={"padding": "20px", "backgroundColor": "#110f0f"})


# Callbacks
@app.callback(Output("line-chart-hist", "figure"), Input("metric-dropdown-hist", "value"))
def update_hist(selected_metric):
    filtered = df_hist[df_hist["Metric"] == selected_metric]
    fig = px.line(filtered, x="Quarter", y="Value",
                  title=f"{selected_metric} Over Time (Historical)",
                  template="plotly_dark")
    fig.update_traces(line=dict(color=color_map_forecast.get(selected_metric, "cyan")))
    fig.update_layout(plot_bgcolor="#1b1b1c",
                      xaxis=dict(tickangle=-45, tickmode="array",
                                 tickvals=[q for q in filtered["Quarter"].unique() if q.endswith("Q4")]))
    return fig

@app.callback([Output("line-chart-forecast", "figure"), Output("kpi-forecast", "children")],
              Input("metric-dropdown-forecast", "value"))
def update_forecast(selected_metric):
    filtered = df_forecast[df_forecast["Metric"] == selected_metric]
    fig = px.line(filtered, x="Quarter", y="Value",
                  title=f"{selected_metric} Forecast (2025–2040)",
                  template="plotly_dark")
    fig.update_traces(line=dict(color=color_map_forecast.get(selected_metric, "cyan")))
    fig.update_layout(plot_bgcolor="#1b1b1c",
                      xaxis=dict(tickangle=-45, tickmode="array",
                                 tickvals=[q for q in filtered["Quarter"].unique() if q.endswith("Q4")]))
    return fig, kpi_block(df_forecast, selected_metric, color_map_forecast.get(selected_metric, "cyan"))

@app.callback([Output("line-chart-scenarios", "figure"), Output("kpi-scenarios", "children")],
              Input("metric-dropdown-scenarios", "value"))
def update_scenarios(selected_metric):
    filtered = df_scenarios[df_scenarios["Metric"] == selected_metric]
    filtered = filtered.groupby("Quarter", as_index=False)["Value"].mean()
    fig = px.line(filtered, x="Quarter", y="Value",
                  title=f"{selected_metric} Forecast (2025–2040)",
                  template="plotly_dark")
    fig.update_traces(line=dict(color=color_map_scenarios.get(selected_metric, "orange")))
    fig.update_layout(plot_bgcolor="#1b1b1c",
                      xaxis=dict(tickangle=-45, tickmode="array",
                                 tickvals=[q for q in filtered["Quarter"].unique() if q.endswith("Q4")]))
    return fig, kpi_block(df_scenarios, selected_metric, color_map_scenarios.get(selected_metric, "orange"))

@app.callback([Output("line-chart-chargers", "figure"), Output("kpi-chargers", "children")],
              Input("metric-dropdown-chargers", "value"))
def update_chargers(selected_metric):
    if selected_metric == "Historical Chargers":
        filtered = df_chargers_hist
    elif selected_metric == "Baseline Forecast":
        filtered = df_chargers_baseline
    else:
        filtered = df_chargers_policy

    fig = px.line(filtered, x="Quarter", y="Value",
                  title=f"{selected_metric} (2022–2040)",
                  template="plotly_dark")
    fig.update_traces(line=dict(color=color_map_chargers.get(selected_metric, "white")))
    fig.update_layout(plot_bgcolor="#1b1b1c",
                      xaxis=dict(tickangle=-45, tickmode="array",
                                 tickvals=[q for q in filtered["Quarter"].unique() if q.endswith("Q4")]))

    if selected_metric == "2030 Target + BEV Sales Regressor Forecast":
        target_value = 300000
        fig.add_scatter(x=["2030Q1", "2040Q4"], y=[target_value, target_value],
                        mode="lines", line=dict(color="red", dash="dash"),
                        name="Govt Target 2030: 300k")

    return fig, kpi_block(pd.concat([df_chargers_hist, df_chargers_baseline, df_chargers_policy]),
                          selected_metric, color_map_chargers.get(selected_metric, "white"))

# Running the dashboard

To view the dashboard inline in Jupyter(or Jupyter VS Code), please run this code below (uncomment the line of code below):

In [4]:

# For viewing the dashboard inline in Jupyter/VS Code, please use (uncomment the line of code below):
#app.run_server(mode="inline")


To view the dashboard in an external browser, execute the block of code below.

In [5]:
# For viewing the dashboard in an external browser instead, uncomment the block below(and comment out the above line of code).  
# By default it will run on port=8050. If that port is already in use on your system,  
# please change it to another (e.g., port=8060).  
#
if __name__ == "__main__":
    print("Dashboard running at: http://127.0.0.1:8050")
    app.run_server(debug=True, port=8050,use_reloader=False)
    # Example alternative:
    # app.run_server(debug=True, port=8060)

Dashboard running at: http://127.0.0.1:8050
